In [1]:
import boto3
from rich import print as rprint
import acore_server_metadata.api as acore_server_metadata

boto_ses = boto3.session.Session(profile_name="bmt_app_dev_us_east_1")
ec2_client = boto_ses.client("ec2")
rds_client = boto_ses.client("rds")

## 获取服务器对象

In [ ]:
sbx_green_server = acore_server_metadata.Server.get_server("sbx-green", ec2_client, rds_client)

In [ ]:
# 如果你的运行环境是在 EC2 里面, 你还可以用这个 API 自己通过检查自己所在的环境获知服务器信息
# acore_server_metadata.Server.from_ec2_inside()

## 获取服务器的基本信息

例如我们可以获得 EC2 和 RDS 的 ID, 以及它们的运行状态. 我们甚至还可以获得游戏服务器的状态, 在线人数等信息. 这些信息是由一个在 EC2 上不断运行的 [守护程序](https://github.com/MacHu-GWU/acore_server_bootstrap-project/blob/main/acore_server_bootstrap/actions/s3_check_server_status/server_monitor.sh) 不断查询游戏数据库信息获得并写入到 EC2 Tag 中的. 我们调用 ``server_lifecycle`` 等属性本质上就是去查询 EC2 Tag 中的信息而已.

In [6]:
print(f"{sbx_green_server.id = }")
print(f"{sbx_green_server.ec2_inst.id = }")
print(f"{sbx_green_server.ec2_inst.status = }")
print(f"{sbx_green_server.rds_inst.id = }")
print(f"{sbx_green_server.rds_inst.status = }")
print(f"{sbx_green_server.server_lifecycle = }")
print(f"{sbx_green_server.wow_status = }")
print(f"{sbx_green_server.wow_status_measure_time = }")

sbx_green_server.id = 'sbx-green'
sbx_green_server.ec2_inst.id = 'i-043cd58afbc720a48'
sbx_green_server.ec2_inst.status = 'running'
sbx_green_server.rds_inst.id = 'sbx-green'
sbx_green_server.rds_inst.status = 'available'
sbx_green_server.server_lifecycle = None
sbx_green_server.wow_status = '0 players'
sbx_green_server.wow_status_measure_time = datetime.datetime(2024, 6, 14, 20, 59, 21, 10695, tzinfo=datetime.timezone.utc)


## 获取服务器的状态

- exists: 指的是服务器存不存在, 如果 EC2 和 RDS 任意一个不存在就视为不存在.
- running: 指的是服务器是否在运行中, 如果 EC2 和 RDS 任意一个不是 running 就视为不在运行中.

In [8]:
print(f"{sbx_green_server.is_exists() = }")
print(f"{sbx_green_server.is_running() = }")
print(f"{sbx_green_server.is_ec2_exists() = }")
print(f"{sbx_green_server.is_ec2_running() = }")
print(f"{sbx_green_server.is_rds_exists() = }")
print(f"{sbx_green_server.is_rds_running() = }")

sbx_green_server.is_exists() = True
sbx_green_server.is_running() = True
sbx_green_server.is_ec2_exists() = True
sbx_green_server.is_ec2_running() = True
sbx_green_server.is_rds_exists() = True
sbx_green_server.is_rds_running() = True


## 重新获取服务器的状态

你可以用 ``.refresh()`` 方法重新获取服务器的状态.

In [10]:
sbx_green_server.refresh(ec2_client, rds_client)

## 批量获取服务器的对象

``.batch_get_server`` 方法进行了一些优化, 可以减少 AWS API 调用次数.

In [13]:
server_mapper = acore_server_metadata.Server.batch_get_server(
    ids=["sbx-green"],
    ec2_client=ec2_client,
    rds_client=rds_client,
)
for server_id, server in server_mapper.items():
    rprint(f"{server_id = }, {server.is_running() = }") 

server_id = 'sbx-green', server.is_running() = True

## 对服务器进行操作

In [14]:
# 创建新的 server 对象
server = acore_server_metadata.Server(id="sbx-green")